### This notebook focues on analysing the London Crime dataset, found on BigQuery. 
### SQL queries will be performed to extract data from the dataset, and then visualised through python libraries. The general analysis and questions asked are merely to practice sql and vizualtion tools, hence the questions may not have real life application or meaning, and therefore present little value.

In [61]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [62]:
from google.cloud import bigquery
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import json

# Create a "Client" object
crime = bigquery.Client()

dataset_ref = crime.dataset("london_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = crime.get_dataset(dataset_ref)

# Construct a reference to the "ga_sessions_20170801" table
table_ref = dataset_ref.table("crime_by_lsoa")

# API request - fetch the table
table = crime.get_table(table_ref)



Using Kaggle's public dataset BigQuery integration.


In [63]:
preview_of_table = """
                SELECT * from `bigquery-public-data.london_crime.crime_by_lsoa`
                ORDER BY major_category DESC
                LIMIT 5
                """
preview_of_table = crime.query(preview_of_table).result().to_dataframe()
preview_of_table

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01000416,Bexley,Violence Against the Person,Murder,0,2008,3
1,E01000554,Brent,Violence Against the Person,Murder,0,2008,1
2,E01000286,Barnet,Violence Against the Person,Murder,0,2008,4
3,E01000885,Camden,Violence Against the Person,Murder,0,2008,3
4,E01000203,Barnet,Violence Against the Person,Murder,0,2008,9


In [64]:
crime_count = """
SELECT SUM(value) as number_of_crimes, borough from `bigquery-public-data.london_crime.crime_by_lsoa` 
GROUP BY borough ORDER BY number_of_crimes DESC;
"""

crime_count = crime.query(crime_count).result().to_dataframe()
crime_count

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,number_of_crimes,borough
0,455028,Westminster
1,292178,Lambeth
2,278809,Southwark
3,275147,Camden
4,262024,Newham
5,260294,Croydon
6,251562,Ealing
7,230286,Islington
8,228613,Tower Hamlets
9,227551,Brent


In [65]:
fig = px.bar(crime_count, x="number_of_crimes", y="borough", color="number_of_crimes", text_auto='.2s', title="Total Number of Crimes in Each Borough", width=1700, height=1000)
fig.update_traces(textfont_size=20, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(yaxis={'categoryorder':'total ascending'})

## Next I wanted to see the year on year crime rate


In [66]:
crime_rate_year = """
SELECT DISTINCT year, sum(value) as total_crime
from
`bigquery-public-data.london_crime.crime_by_lsoa`
where value =1
group by year;
"""

crime_rate_year = crime.query(crime_rate_year).result().to_dataframe()
crime_rate_year

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,year,total_crime
0,2008,240966
1,2009,238097
2,2010,235513
3,2011,234747
4,2012,234714
5,2013,232720
6,2014,237515
7,2015,241347
8,2016,246170


In [67]:
fig = px.line(crime_rate_year, x="year", y="total_crime", text='total_crime', title="London Crime Totals (year on year)", width=1800, height=500, )
fig.update_traces(textposition="top center", textfont_size=15)
fig.show()

## Next, I wanted to see the most common type of crime

In [68]:
types_of_crime = """
SELECT DISTINCT minor_category as crime_type, SUM(value) as total
from `bigquery-public-data.london_crime.crime_by_lsoa`
GROUP BY crime_type ORDER BY total DESC;
"""

types_of_crime = crime.query(types_of_crime).result().to_dataframe()
types_of_crime

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,crime_type,total
0,Other Theft,980085
1,Theft From Motor Vehicle,569956
2,Burglary in a Dwelling,491282
3,Harassment,458124
4,Assault with Injury,451001
5,Possession Of Drugs,431948
6,Common Assault,413690
7,Theft From Shops,345142
8,Other Theft Person,308842
9,Criminal Damage To Motor Vehicle,265463


### As 'Other theft' could not be easily identified or described, I focused on the next highest offence: 'Theft From Motor Vehicle'.
### I wanted to see how this type of crime changed over the years

In [69]:
theft_from_mvh = """
SELECT minor_category, SUM(value) as total, year
from `bigquery-public-data.london_crime.crime_by_lsoa`
WHERE minor_category = 'Theft From Motor Vehicle'
GROUP BY minor_category, year
"""

theft_from_mvh = crime.query(theft_from_mvh).result().to_dataframe()
theft_from_mvh

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,total,year
0,Theft From Motor Vehicle,76570,2008
1,Theft From Motor Vehicle,68856,2009
2,Theft From Motor Vehicle,69709,2010
3,Theft From Motor Vehicle,69604,2011
4,Theft From Motor Vehicle,68198,2012
5,Theft From Motor Vehicle,63509,2013
6,Theft From Motor Vehicle,52167,2014
7,Theft From Motor Vehicle,50024,2015
8,Theft From Motor Vehicle,51319,2016


In [70]:
fig = px.bar(theft_from_mvh, x="year", y="total", text_auto='.2s', title="Theft from a Motor Vehicle - Crime Totals, by year", width=1800, height=500, )
fig.update_traces(textfont_size=15, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

### I looked at the number of theft from a motor vehicle by month, for the year 2008.

In [71]:
month_car = """
SELECT SUM(value) as motor_veh_theft, month,
CASE WHEN month = 1 then "Jan"
     WHEN month = 2 then "Feb"
     WHEN month = 3 then "Mar"
     WHEN month = 4 then "Apr"
     WHEN month = 5 then "May"
     WHEN month = 6 then "Jun"
     WHEN month = 7 then "Jul"
     WHEN month = 8 then "Aug"
     WHEN month = 9 then "Sep"
     WHEN month = 10 then "Oct"
     WHEN month = 11 then "November"
     WHEN month = 12 then "December"
ELSE "no month"  END AS actual_month 
from `bigquery-public-data.london_crime.crime_by_lsoa`
WHERE minor_category = 'Theft From Motor Vehicle' and year = 2008
GROUP BY month
ORDER BY motor_veh_theft desc
"""

month_car = crime.query(month_car).result().to_dataframe()
month_car

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,motor_veh_theft,month,actual_month
0,7123,1,Jan
1,6901,3,Mar
2,6785,10,Oct
3,6552,4,Apr
4,6488,2,Feb
5,6373,11,November
6,6271,9,Sep
7,6188,5,May
8,6092,7,Jul
9,5988,12,December


## Now to move away from single types of crime, I wanted to explore the single worst borough for crime, for each year.

In [72]:
worst_bor = """
select MAX(crime_rate) as number_offences,
 year, 
 borough,
RANK() OVER(PARTITION BY year ORDER BY MAX(crime_rate) desc) Rank

from 
(select year, 
borough,
sum(value) as crime_rate

from `bigquery-public-data.london_crime.crime_by_lsoa`
group by year, borough)


group by year, borough
order by Rank, number_offences desc
limit 9;
"""
#I then had to create a copy, so i could order by year
worst_bor = crime.query(worst_bor).result().to_dataframe()
worst_bor2 = worst_bor.copy()
worst_bor2.sort_values('year')

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,number_offences,year,borough,Rank
6,47846,2008,Westminster,1
3,48456,2009,Westminster,1
5,48116,2010,Westminster,1
1,54243,2011,Westminster,1
0,59226,2012,Westminster,1
2,53852,2013,Westminster,1
7,47564,2014,Westminster,1
8,47395,2015,Westminster,1
4,48330,2016,Westminster,1


#### Westminster was the single worst borough for each year!

## Now lets look at Murder percentages difference from murders in 2008 to murders in 2016, for each borough.

In [73]:
murder_rate = """
SELECT murder_2008, murder_2016, borough,
(murder_2016 - murder_2008) AS change,
round((NULLIF(murder_2016,0) - NULLIF(murder_2008,0)) / murder_2016 * 100, 2) as perc_change


from

(select 

borough,
SUM(IF(year = 2008, value, NULL)) AS murder_2008,
SUM(IF(year = 2016, value, NULL)) AS murder_2016
from 
`bigquery-public-data.london_crime.crime_by_lsoa`
where minor_category = "Murder"
group by borough)

order by perc_change desc;
"""

murder_rate = crime.query(murder_rate).result().to_dataframe()
murder_rate

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,murder_2008,murder_2016,borough,change,perc_change
0,2,6,Camden,4,66.67
1,1,3,Hammersmith and Fulham,2,66.67
2,2,4,Greenwich,2,50.00
3,1,2,Havering,1,50.00
4,5,9,Croydon,4,44.44
5,3,5,Haringey,2,40.00
6,3,5,Hillingdon,2,40.00
7,4,5,Bexley,1,20.00
8,4,5,Tower Hamlets,1,20.00
9,5,6,Newham,1,16.67


In [74]:
fig = px.bar(murder_rate, x="change", y="borough", text_auto='.2s', title="Murder difference in standalone year: 2008 and 2016", width=700, height=700, orientation='h')
fig.update_traces(textfont_size=15, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

### I wanted to see if any borough's had 0 recorded murders

In [75]:
no_murder = """
Select DISTINCT borough, 
from `bigquery-public-data.london_crime.crime_by_lsoa`

WHERE borough NOT IN 

(SELECT borough 
             from `bigquery-public-data.london_crime.crime_by_lsoa`
             where minor_category = "Murder" AND value >0)
"""

no_murder = crime.query(no_murder).result().to_dataframe()
no_murder

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,borough
0,City of London


### City of London was the only borough to record 0 murders.
### See below all other boroughs and their total murders:


In [76]:
borough_murder = """

SELECT borough, sum(value) as number_of_murders
from `bigquery-public-data.london_crime.crime_by_lsoa`
 where minor_category = "Murder"
 GROUP BY borough
 ORDER BY number_of_murders desc


"""

borough_murder = crime.query(borough_murder).result().to_dataframe()
borough_murder

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,borough,number_of_murders
0,Newham,56
1,Lambeth,53
2,Southwark,52
3,Brent,47
4,Ealing,46
5,Croydon,45
6,Islington,45
7,Tower Hamlets,45
8,Hackney,44
9,Haringey,44


In [77]:

fig = px.pie(borough_murder, values='number_of_murders', names='borough', title='Murders per borough')
fig.show()

### I wanted to see if there were any particular months that had murder rates, equal to 10 or over.

In [78]:
month_murder_over_10 = """
SELECT SUM(value) as total, borough, month
from `bigquery-public-data.london_crime.crime_by_lsoa`
where minor_category = 'Murder'
group by month, borough
having total >= 10
"""

month_murder_over_10 = crime.query(month_murder_over_10).result().to_dataframe()
month_murder_over_10

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,total,borough,month
0,10,Islington,6


### I wanted to see the two worst months and boroughs for crime in the whole dataset.

In [79]:
worst_month = """
SELECT SUM(value) as total, borough, month, year
from `bigquery-public-data.london_crime.crime_by_lsoa`
group by borough, year, month
order by total desc
limit 2

"""

worst_month =  crime.query(worst_month).result().to_dataframe()
worst_month

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,total,borough,month,year
0,5428,Westminster,7,2012
1,5362,Westminster,3,2012


### It turns out that July 2012 was the worst recorded month for crimes. This happened to be when the London Olympics were being hosted.
### Let's look at what types of crime were occuring then:

In [80]:
two_worst_crime = """

SELECT minor_category, july_2012, march_2012, (july_2012 - march_2012) as difference

FROM 
(select minor_category,
SUM(IF(year = 2012 and month = 7, value, NULL)) AS july_2012,
SUM(IF(year = 2016 and month = 3, value, NULL)) AS march_2012
from 
`bigquery-public-data.london_crime.crime_by_lsoa`
where borough = 'Westminster'
group by minor_category)
order by difference desc
"""

two_worst_crime = crime.query(two_worst_crime).result().to_dataframe()
two_worst_crime

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning:

Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.



,minor_category,july_2012,march_2012,difference
0,Other Theft,1691,925,766
1,Other Theft Person,870,442,428
2,Possession Of Drugs,397,133,264
3,Theft/Taking of Pedal Cycle,238,83,155
4,Personal Property,223,156,67
5,Theft From Motor Vehicle,245,180,65
6,Criminal Damage To Motor Vehicle,106,50,56
7,Drug Trafficking,65,13,52
8,Assault with Injury,212,161,51
9,Theft From Shops,394,346,48


In [81]:
new_two_worst = two_worst_crime.copy()
crime_list = []
for i in new_two_worst.minor_category:
    crime_list.append(i)



In [82]:
july_list = []
for i in new_two_worst.july_2012:
    july_list.append(i)

In [83]:
march_list = []
for i in new_two_worst.march_2012:
    march_list.append(i)

### I wanted to compare the the 2 worst months of crime reports, based on type of crime

In [85]:

import plotly.graph_objects as go
x = crime_list
fig = go.Figure(data=[
    go.Bar(name='july_2012', x=crime_list, y=july_list),
    go.Bar(name='march_2012', x=crime_list, y=march_list)
])
# Change the bar mode
fig.update_layout(barmode='group')



fig.show()